In [ ]:
# --- Configuration & File Path Setup ---
import os
from IPython.display import display, Markdown  # Jupyter-specific display tools

# Create temp directory if needed
TEMP_DIR = os.path.join(os.getcwd(), 'temp')
os.makedirs(TEMP_DIR, exist_ok=True)

# List files starting with 'df_OHLCV_' in TEMP_DIR
matching_files = [f for f in os.listdir(TEMP_DIR) if f.startswith('df_OHLCV_')]

if not matching_files:
    display(Markdown("**Error:** No matching files found in temp directory!"))
    exit()

# Display files with bold headers using Markdown
display(Markdown("**Available files:**"))
for idx, filename in enumerate(matching_files, 1):
    display(Markdown(f"- ({idx}) `{filename}`"))

# Jupyter-optimized input handling
while True:
    try:
        print(f'\nInput a number into the prompt (top of the screen) to select file to process')
        # Create a prominent input prompt
        choice = int(input(f"\n\033[1mEnter choice (1-{len(matching_files)}): \033[0m"))
        if 1 <= choice <= len(matching_files):
            SOURCE_FILENAME = matching_files[choice - 1]
            break
        else:
            display(Markdown(f"<span style='color:red'>Invalid: Enter 1-{len(matching_files)}</span>"))
    except ValueError:
        display(Markdown("<span style='color:red'>Numbers only please!</span>"))

# Generate cleaned filename
base, ext = os.path.splitext(SOURCE_FILENAME)
CLEANED_FILENAME = f"{base}_clean{ext}"

# Build paths and show confirmation
source_path = os.path.join(TEMP_DIR, SOURCE_FILENAME)
dest_path = os.path.join(TEMP_DIR, CLEANED_FILENAME)

display(Markdown(f"""
**Selected paths:**
- Source: `{source_path}`  
- Destination: `{dest_path}`
"""))

In [ ]:
# --- Data Loading & Initial Inspection ---
import pandas as pd

# Load raw data from pickle file
df = pd.read_pickle(source_path)

# Display initial data structure
print("[Raw Data Overview]")
display(df.head())
df.info()

In [ ]:
# --- Data Filtering & Cleaning ---
import utils  # Custom utility functions

# 1. Align dates across all symbols using SPY as reference
df = utils.filter_df_dates_to_reference_symbol(df=df, reference_symbol='SPY')

# 2. Remove symbols with missing data points
df_clean, missing_symbols = utils.filter_symbols_with_missing_values(df)

# Display cleaning results
print("\n[Cleaning Report]")
print(f"Removed {len(missing_symbols)} symbols with missing data: {missing_symbols}")
print("\n[Cleaned Data Structure]")
df_clean.info()

In [ ]:
# --- Save Cleaned Data ---
# Save processed data to pickle file
df_clean.to_pickle(dest_path)
print(f"\n[Save Successful] Cleaned data saved to:\n{dest_path}")